In [289]:
import os

import geojson
import json
from shapely.geometry import shape, asMultiPoint, asPolygon, Polygon, MultiPolygon
from shapely import wkt
from shapely.ops import cascaded_union, unary_union

from geojson import Feature, Point, FeatureCollection

from build_dstl import *
from core_dstl import *


In [209]:
# get gridsize for subregion from json
with open("../input/train_geojson_v3/6100_1_3/Grid_6010.geojson") as fn:
    data = json.load(fn)

    feature_collection = FeatureCollection(data['features'])
    grid_size_coordinates = feature_collection["features"][0]["geometry"]["coordinates"]
print (grid_size_coordinates)


[[[0.009156, 0], [0.009156, -0.009042], [0, -0.009042], [0, 0], [0.009156, 0]]]


In [210]:
json_imageIds = json_area_regions
test_file_prefix = "../input/train_geojson_v3/"
file_type = ".geojson"
json_filenames =[file_in.replace(file_type, "") for file_in in os.listdir(test_file_prefix+"6100_1_3")]

In [316]:
def load_geojson_polygons(imageIds):
    """ Load polygons from a geojson file
    
    returns a dictionary (imageIds) of dictionaries (class_types) of polygons
    """ 
    
    if imageIds is None : imageIds = td.imageId.unique()
    
    
    polygons = {}
    for iid in imageIds:
        df_image = iid
        classPolygons = {}
        
        # get json filenames per directory, not all the directories have the same files
        filenames = [file.replace(file_type, '') for file in os.listdir(test_file_prefix+iid)]
        
        for json_file in filenames:
            with open(test_file_prefix + iid +"/"+ json_file + file_type) as fn:
                data = json.load(fn)

                feature_collection = FeatureCollection(data['features'])

#                 print first json feature of iid file for sanity check
#                 print feature_collection['features'][0]
        
                num_features = len(feature_collection["features"])

                multipolygons = []
                for index in range(num_features):

                    progress()

                    if json_file in filename_to_classType:
                        filename = json_file
                    else:
                        filename = "extended_classes"         


                    ctype = filename_to_classType[filename]

                    multipolygon = shape(feature_collection['features'][index]['geometry'])


        #             # At least one polygon in the training data is invalid. Fix (Kudos: @amanbh)
        #             # https://www.kaggle.com/amanbh/dstl-satellite-imagery-feature-detection/dealing-with-invalid-polygons    
                    if not multipolygon.is_valid: 
                        multipolygon = multipolygon.buffer(0)
                        progress('Fixed invalid multipolygon {} {}'.format(iid, ctype) )

                    if not multipolygon.is_empty:
                        multipolygons.append(multipolygon)

  
                if ctype not in classPolygons:
                    classPolygons[ctype] = unary_union(MultiPolygon(list(multipolygons)))
                    print ctype, "added to classPolygons dictionary", classPolygons[ctype].area
                    

                else:
                    print ctype, "is already in dictionary"
                    polygons_list = [classPolygons.get(ctype, "{} has no polygons".format(ctype))]
                    polygons_list.extend(multipolygons)
                    
                    print ctype, [p.is_valid for p in polygons_list]
                    classPolygons[ctype] = unary_union(MultiPolygon(polygons_list))
                    print ctype, "has new polygons", classPolygons[ctype].area



            print ctype, "geojson to polygon ---> success \n"

    
        polygons[iid] = classPolygons
        print polygons.keys()

                
    progress("Done!")

#     print polygons
    return polygons


json_polygon_dict = load_geojson_polygons(["6100_1_3"])

. 3 added to classPolygons dictionary 1.834404e-06
3 geojson to polygon ---> success 

.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [288]:
from pprint import pprint 

pprint (json_polygon_dict.keys())


# for polygons in json_polygon_dict['6100_1_3'][15]:
#     print class_num, type(polygons.wkt)

print [p.is_valid for p in json_polygon_dict['6100_1_3'][10]]



['6100_1_3']
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [306]:
def json_polygons_to_composite(class_polygons, xmax, ymin, width, height, filename, outline=False):
    """ If outline is true, create transparent outline of classes suitable for layering over other images."""

    image_ids = class_polygons.keys()
    print image_ids
    
    for iid in image_ids:
        
        width /= float(dpi)
        height /= float(dpi)

        if outline :
            linewidth = 0.2
            transparent = True
            fill = False
        else:
            linewidth = 0.0
            transparent = False
            fill = True
        
        fig = plt.figure(figsize=(width,height), frameon=False)
        axes = plt.Axes(fig, [0., 0, 1, 1]) # One axis, many axes
        axes.set_axis_off()         
        fig.add_axes(axes)
        
    
        for classType, multipolygon in class_polygons[iid].items():

            print classType, type(multipolygon), multipolygon.area  
            
            if classType != 15:

                if (isinstance(multipolygon, shapely.geometry.multipolygon.MultiPolygon)):
                    for polygon in multipolygon:
                        patch = PolygonPatch(polygon,
                                             color=class_color[classType],
                                             lw=linewidth,   
                                             alpha=1.0,
                                             zorder=class_zorder[classType],
                                             antialiased =True,
                                             fill = fill)  

                        axes.add_patch(patch)


                else:
                    patch = PolygonPatch(multipolygon,
                                 color=class_color[classType],
                                 lw=linewidth,   
                                 alpha=1.0,
                                 zorder=class_zorder[classType],
                                 antialiased =True,
                                 fill = fill)

                    axes.add_patch(patch)


        axes.set_xlim(0, xmax)
        axes.set_ylim(ymin, 0)
        axes.set_aspect(1)
        plt.axis('off')

        filename = str(iid) + "_"+filename

        plt.savefig(filename, pad_inches=0, dpi=dpi, transparent=transparent)
        plt.clf()
        plt.close()    




In [307]:
image_ids = json_polygon_dict.keys()
print image_ids
iid = image_ids[0]
print json_polygon_dict[iid][15]

 

#     for classType, multipolygon in json_dict[iid].items():
#         print classType, type(multipolygon), multipolygon.length, multipolygon.area
    
json_polygons_to_composite(json_polygon_dict, std_xmax, std_ymin, std_width, 
    std_height, "test_test_test_json_", outline=False)


['6100_1_3']
POLYGON ((0.009155999999999999 0, 0.009155999999999999 -0.009042, 0 -0.009042, 0 0, 0.009155999999999999 0))
['6100_1_3']
1 <class 'shapely.geometry.multipolygon.MultiPolygon'> 3.85348e-07
2 <class 'shapely.geometry.multipolygon.MultiPolygon'> 2.26053171775e-06
3 <class 'shapely.geometry.polygon.Polygon'> 1.834404e-06
4 <class 'shapely.geometry.multipolygon.MultiPolygon'> 2.03170444066e-06
5 <class 'shapely.geometry.multipolygon.MultiPolygon'> 3.92614454902e-06
6 <class 'shapely.geometry.multipolygon.MultiPolygon'> 2.228386e-06
8 <class 'shapely.geometry.multipolygon.MultiPolygon'> 5.665135e-07
9 <class 'shapely.geometry.multipolygon.MultiPolygon'> 9.109e-09
10 <class 'shapely.geometry.multipolygon.MultiPolygon'> 3.34628965723e-08
15 <class 'shapely.geometry.polygon.Polygon'> 8.2788552e-05
